In [68]:
import requests

In [69]:
result=requests.get('https://quotes.toscrape.com/')

In [70]:
type(result)

requests.models.Response

In [71]:
result.text

'<!DOCTYPE html>\n<html lang="en">\n<head>\n\t<meta charset="UTF-8">\n\t<title>Quotes to Scrape</title>\n    <link rel="stylesheet" href="/static/bootstrap.min.css">\n    <link rel="stylesheet" href="/static/main.css">\n</head>\n<body>\n    <div class="container">\n        <div class="row header-box">\n            <div class="col-md-8">\n                <h1>\n                    <a href="/" style="text-decoration: none">Quotes to Scrape</a>\n                </h1>\n            </div>\n            <div class="col-md-4">\n                <p>\n                \n                    <a href="/login">Login</a>\n                \n                </p>\n            </div>\n        </div>\n    \n\n<div class="row">\n    <div class="col-md-8">\n\n    <div class="quote" itemscope itemtype="http://schema.org/CreativeWork">\n        <span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>\n        <sp

In [72]:
import bs4

In [73]:
soup = bs4.BeautifulSoup(result.text, "lxml")


In [74]:
soup

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Quotes to Scrape</title>
<link href="/static/bootstrap.min.css" rel="stylesheet"/>
<link href="/static/main.css" rel="stylesheet"/>
</head>
<body>
<div class="container">
<div class="row header-box">
<div class="col-md-8">
<h1>
<a href="/" style="text-decoration: none">Quotes to Scrape</a>
</h1>
</div>
<div class="col-md-4">
<p>
<a href="/login">Login</a>
</p>
</div>
</div>
<div class="row">
<div class="col-md-8">
<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="t

In [75]:
soup.select(".author")

[<small class="author" itemprop="author">Albert Einstein</small>,
 <small class="author" itemprop="author">J.K. Rowling</small>,
 <small class="author" itemprop="author">Albert Einstein</small>,
 <small class="author" itemprop="author">Jane Austen</small>,
 <small class="author" itemprop="author">Marilyn Monroe</small>,
 <small class="author" itemprop="author">Albert Einstein</small>,
 <small class="author" itemprop="author">André Gide</small>,
 <small class="author" itemprop="author">Thomas A. Edison</small>,
 <small class="author" itemprop="author">Eleanor Roosevelt</small>,
 <small class="author" itemprop="author">Steve Martin</small>]

In [76]:
soup.select(".text")

[<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>,
 <span class="text" itemprop="text">“It is our choices, Harry, that show what we truly are, far more than our abilities.”</span>,
 <span class="text" itemprop="text">“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”</span>,
 <span class="text" itemprop="text">“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”</span>,
 <span class="text" itemprop="text">“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”</span>,
 <span class="text" itemprop="text">“Try not to become a man of success. Rather become a man of value.”</span>,
 <span class="text" itemprop="text">“It is better to be hated for what you are than to be loved for what you are not.

In [77]:
quotes = []

for x in soup.select('.text'):
    quotes.append(x.text)

quotes

['“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
 '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
 '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
 '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
 "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",
 '“Try not to become a man of success. Rather become a man of value.”',
 '“It is better to be hated for what you are than to be loved for what you are not.”',
 "“I have not failed. I've just found 10,000 ways that won't work.”",
 "“A woman is like a tea bag; you never know how strong it is until it's in hot water.”",
 '“A day without sunshine is like, you know, night.”']

In [78]:

len(soup.select('.tag-item'))

10

In [79]:
for item in soup.select('.tag-item'):
    print(item.text)


love


inspirational


life


humor


books


reading


friendship


friends


truth


simile



In [80]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def scrape_ft_bitcoin_articles():
    base_url = 'https://www.ft.com/bitcoin'
    page_number = 1
    article_dates = []
    article_titles = []

    while True:
        url = f'{base_url}?page={page_number}'
        result = requests.get(url)
        soup = BeautifulSoup(result.text, 'html.parser')
        dates = soup.select('time.o-date')
        titles = soup.select('a.js-teaser-heading-link')

        for date, title in zip(dates, titles):
            article_dates.append(date.get('datetime'))
            article_titles.append(title.get_text(strip=True))

        next_page_link = soup.find('a', class_='o-buttons-icon--arrow-right')
        if not next_page_link:
            break

        page_number += 1

    df = pd.DataFrame({
        'Date': article_dates,
        'Title': article_titles
    })

    # Convert the 'Date' column to timestamps
    df['Date'] = pd.to_datetime(df['Date'])

    # Filter articles from the last 3 months
    today = pd.to_datetime('today')
    df['Date'] = df['Date'].dt.tz_localize(None)
    three_months_ago = today - pd.DateOffset(months=3)
    filtered_df = df[df['Date'] >= three_months_ago]


    return filtered_df

# Call the function and print the filtered DataFrame
bitcoin_articles_df = scrape_ft_bitcoin_articles()
print(bitcoin_articles_df)


                      Date                                              Title
0  2024-05-30 06:00:05.535     European bitcoin ETPs suffer mounting outflows
1  2024-05-24 14:37:18.548  British-Chinese bitcoin money launderer jailed...
2  2024-05-24 12:28:10.544                      Cryptofinance: into the ether
3  2024-05-23 22:30:14.553  SEC paves way for ethereum ETFs in boost for c...
4  2024-05-22 11:51:01.390           First UK crypto ETPs to launch on May 28
5  2024-05-21 11:14:59.756     Ether jumps 16% on US ETF approval speculation
6  2024-05-20 16:56:20.240  Head of crypto asset manager Grayscale steps down
7  2024-05-20 16:29:28.157  Edited highlights from the evisceration of Cra...
8  2024-05-17 12:00:56.082            My political enemy’s enemy is my friend
9  2024-05-16 04:00:58.416  Futures exchange CME plans to launch bitcoin t...
10 2024-05-14 04:00:45.376  BlackRock closes in on crown of world’s larges...
11 2024-05-13 11:00:54.163               The paradox of ‘sustain